In [17]:
from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel

In [18]:
bqm=BinaryQuadraticModel('BINARY')

### Create N-Queen problem matrix $Q$

In [19]:
import numpy as np

N=8

class QuboPoly():
    def __init__(self, n=1024):
        self.array=np.zeros((n, n), dtype=int)
        self.constant=0
        self._size=n
    
    def add_term(self, i, j, c):
        if i>=self._size or j>=self._size:
            raise RuntimeError("Wrong index")
        self.array[i][j]+=c
        
    def add_constant(self, c):
        self.constant+=c
        
    def sum(self, p):
        if self._size != p._size:
            raise RuntimeError("Wrong polynomial size")
        self.array+=p.array
        self.constant+=p.constant
        
    def power(self):
        a=np.diag(self.array)
        self.array=np.outer(a, a) + 2*self.constant*np.diag(a) # convert back to NxN
        self.constant**=2
        
    def multiply(self, p):
        a=np.diag(self.array)
        b=np.diag(p.array)
        self.array=np.outer(a, b) + self.constant*np.diag(b) + p.constant*np.diag(a)
        self.constant*=p.constant
        
def build_column_penalty(N):
    qubo=QuboPoly(N*N)
    for col in range(N):
        tmp=QuboPoly(N*N)
        for i in grid[:, col]:
            tmp.add_term(i, i, 1)
        tmp.add_constant(-1)
        tmp.power()
        qubo.sum(tmp)
    return qubo

def build_row_penalty(N):
    qubo=QuboPoly(N*N)
    for row in range(N):
        tmp=QuboPoly(N*N)
        for i in grid[row]:
            tmp.add_term(i, i, 1)
        tmp.add_constant(-1)
        tmp.power()
        qubo.sum(tmp)
    return qubo

def build_diag_penalty(N):
    qubo=QuboPoly(N*N)
    for g in [grid, np.fliplr(grid)]:
        for k in range(-N+2, N-1):
            tmp1=QuboPoly(N*N)
            tmp2=QuboPoly(N*N)
            for dia in np.diag(g, k=k):
                tmp1.add_term(dia, dia, 1)
                tmp2.add_term(dia, dia, 1)
            tmp1.add_constant(-1)
            tmp1.multiply(tmp2)
            qubo.sum(tmp1)
    return qubo

grid=np.arange(N*N).reshape(N, N)
P1=build_column_penalty(N)
P2=build_row_penalty(N)
P3=build_diag_penalty(N)
qubo_obj=QuboPoly(N*N)
qubo_obj.sum(P1)
qubo_obj.sum(P2)
qubo_obj.sum(P3)
print(qubo_obj.array)
print(qubo_obj.constant)

[[-2  1  1 ...  0  0  1]
 [ 1 -2  1 ...  0  0  0]
 [ 1  1 -2 ...  0  0  0]
 ...
 [ 0  0  0 ... -2  1  1]
 [ 0  0  0 ...  1 -2  1]
 [ 1  0  0 ...  1  1 -2]]
16


In [20]:
from collections import defaultdict
Q=defaultdict(int)

for i in range(qubo_obj._size):
    for j in range(qubo_obj._size):
        Q[(i, j)]=qubo_obj.array[i][j]

In [21]:
bqm=BinaryQuadraticModel.from_qubo(Q)

### Use D-Wave's solver

In [23]:
sampler=EmbeddingComposite(DWaveSampler())
sampleset=sampler.sample(bqm, num_reads=10)

In [29]:
for s, e in sampleset.data(['sample', 'energy']):
    print("Sample:", s, "Energy:", e)

Sample: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 1, 23: 0, 24: 1, 25: 0, 26: 0, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 1, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 1, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0} Energy: -8.0
Sample: {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 1, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 0, 41: 1, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 1, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0} Energy: -8.0
Sample: {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 

In [41]:
first_sample=sampleset.first.sample
first_energy=sampleset.first.energy

#print("First Sample:", first_sample)
print("Energy of First Sample:", first_energy)

Energy of First Sample: -8.0


In [42]:
# map it back to answer
solution=np.zeros(N*N, dtype=int)
for index, var in enumerate(first_sample):
    solution[index]=int(first_sample[var])
    
board_sol=solution.reshape(N, N)
print(board_sol)

[[0 0 0 0 0 0 0 1]
 [0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 1 0]
 [1 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0]]
